Облачный CLEARML

In [3]:

%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=UYT7CN4QVOORGQHYBKIC17B9SQUWOP
%env CLEARML_API_SECRET_KEY=L06BUQ4bFm7nNmd3wHK-kDmC0H6i7HnYqKAOGaKuCCF3cCEi1FTJTGh0dABiOgCQY8c

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=UYT7CN4QVOORGQHYBKIC17B9SQUWOP
env: CLEARML_API_SECRET_KEY=L06BUQ4bFm7nNmd3wHK-kDmC0H6i7HnYqKAOGaKuCCF3cCEi1FTJTGh0dABiOgCQY8c


# Локальный ClearML

In [1]:
%env CLEARML_WEB_HOST=http://neuro.kbl-kr.ru/
%env CLEARML_API_HOST=http://neuro.kbl-kr.ru:8008
%env CLEARML_FILES_HOST=http://neuro.kbl-kr.ru:8081/
%env CLEARML_API_ACCESS_KEY=W5GHI79KG5PVRRULKY63CK9154OTXB
%env CLEARML_API_SECRET_KEY=LL1OCfgO714o693AuAUfEHX9QWUnQ9n-h8en8EbD6oJpa8f8uPCWjXzzvOaaBNCu0h4

env: CLEARML_WEB_HOST=http://neuro.kbl-kr.ru/
env: CLEARML_API_HOST=http://neuro.kbl-kr.ru:8008
env: CLEARML_FILES_HOST=http://neuro.kbl-kr.ru:8081/
env: CLEARML_API_ACCESS_KEY=W5GHI79KG5PVRRULKY63CK9154OTXB
env: CLEARML_API_SECRET_KEY=LL1OCfgO714o693AuAUfEHX9QWUnQ9n-h8en8EbD6oJpa8f8uPCWjXzzvOaaBNCu0h4


In [2]:
import pandas as pd
import numpy as np
from clearml import Task, Logger, Dataset
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ParameterSampler
# from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score

In [3]:
task = Task.init(
    project_name='SmallObjectDetection',
    task_name='FOMO-mva23_train',
    tags=['FOMO'])

ClearML Task: created new task id=2a4882633e4c48dca9d2fa74dad794cb
ClearML results page: http://neuro.kbl-kr.ru/projects/fb88cba1422a4d028da40bd2c7fed04f/experiments/2a4882633e4c48dca9d2fa74dad794cb/output/log
2025-09-18 12:53:13,544 - clearml.Task - INFO - Storing jupyter notebook directly as code
CLEARML-SERVER new package available: UPGRADE to v2.2.0 is recommended!
Release Notes:
## New Features and Improvements

- Update fixed users password note in apiserver.conf (#284, thanks @djiboshin!)
- New UI global search including quick filters ([ClearML #1041](https://github.com/allegroai/clearml/issues/1041))
- Add persistent UI plot properties: Plot settings (e.g. logarithmic/linear scale, hover mode) are retained across project tasks
- Add option to hide original graph when smoothing is enabled in UI plot ([ClearML #1400](https://github.com/clearml/clearml/issues/1400))
- Add persistent UI table details view ([ClearML Web #105](https://github.com/clearml/clearml-web/issues/105)) 
- Ad

In [35]:
task = Task.get_task(task_id='2a4882633e4c48dca9d2fa74dad794cb')

In [32]:
task.close()

# FOMO

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models import mobilenet_v2
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import cv2
import json
import os
from pycocotools.coco import COCO
from tqdm import tqdm
# from torch.utils.tensorboard import SummaryWriter
import time
from clearml import Dataset as CML_Dataset

In [ ]:
task.execute_remotely(queue_name='default', exit_process=True)

Switching to remote execution, output log page http://neuro.kbl-kr.ru/projects/fb88cba1422a4d028da40bd2c7fed04f/experiments/2a4882633e4c48dca9d2fa74dad794cb/output/log
ClearML Terminating local execution process - continuing execution remotely


In [5]:
# --- 0. Dataset ---
# Пути к данным COCO
coco_dataset = CML_Dataset.get(dataset_name="FOMO-mva23", dataset_project="SmallObjectDetection")
dataset_path = coco_dataset.get_local_copy()

# dataset_path = r'C:/Users/ILYA/.clearml/cache/storage_manager/datasets/ds_b8a609f71dd347a59ed298538fbc8195/train/images'

print("Dataset loadeddd")

2025-09-18 12:53:39,787 - clearml - INFO - Dataset.get() did not specify alias. Dataset information will not be automatically logged in ClearML Server.
2025-09-18 12:53:40,918 - clearml.storage - INFO - Downloading: 49.54MB from http://neuro.kbl-kr.ru:8081/SmallObjectDetection/.datasets/FOMO-mva23/FOMO-mva23.b2fb40384bad494d88e1b6340553c4a4/artifacts/data/dataset.b2fb40384bad494d88e1b6340553c4a4.m3ho3y2i.zip


███████████████████████████████ 100% | 49.54/49.54 MB [00:01<00:00, 46.07MB/s]: 


2025-09-18 12:53:42,008 - clearml.storage - INFO - Downloaded 49.54 MB successfully from http://neuro.kbl-kr.ru:8081/SmallObjectDetection/.datasets/FOMO-mva23/FOMO-mva23.b2fb40384bad494d88e1b6340553c4a4/artifacts/data/dataset.b2fb40384bad494d88e1b6340553c4a4.m3ho3y2i.zip , saved to C:/Users/ILYA/.clearml/cache/storage_manager/datasets/f4885cd577703322b7ce61436361380b.dataset.b2fb40384bad494d88e1b6340553c4a4.m3ho3y2i.zip
Dataset loadeddd


In [10]:
TRAIN_ANNOTATION_FILE = f"{dataset_path}/train/train_annotations/mva23_FOMO_train.json"
TRAIN_IMAGE_DIR = f"{dataset_path}/train/images"
VAL_ANNOTATION_FILE = f"{dataset_path}/val/val_annotations/mva23_FOMO_val.json"
VAL_IMAGE_DIR = f"{dataset_path}/val/images"

In [7]:
print(TRAIN_IMAGE_DIR)

C:/Users/ILYA/.clearml/cache/storage_manager/datasets/ds_b8a609f71dd347a59ed298538fbc8195/train/images/train/images


In [8]:
# --- 1. Конфигурация ---

params = {
"NUM_CLASSES" : 2,  # Кол-во классов (включая фон)
"INPUT_SIZE" : (224, 224), # Размер входного изображения
'BATCH_SIZE' : 8,
"EPOCHS" : 10,
"LR" : 1e-3,
"trunkAt" : 4, # Номер слоя, где обрезать MobileNet. Для карты размером 56 это значение 4
}
params = task.connect(params)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# --- 2. Датасет COCO ---
class CocoDataset(Dataset):
    def __init__(self, annotation_file, image_dir, transform=None):
        self.coco = COCO(annotation_file)
        self.image_ids = list(self.coco.imgs.keys())
        self.image_dir = image_dir
        self.transform = transform
        self.cat_ids = self.coco.getCatIds()

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.image_dir, img_info['file_name'])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Получаем аннотации для изображения
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)

        # Создаём маску классов (H, W)
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        for ann in anns:
            if ann['category_id'] in self.cat_ids:
                class_id = self.cat_ids.index(ann['category_id']) + 1 # 0 - фон
                if 'segmentation' in ann:
                    # Если есть segmentation, используем его
                    mask += self.coco.annToMask(ann) * class_id
                else:
                    # Если нет - создаём маску из bbox
                    x, y, w, h = ann['bbox']
                    mask[int(y):int(y + h), int(x):int(x + w)] = class_id
                    # print(f'{class_id=}')

        # Применяем трансформации
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        mask = augmented['mask']  # mask уже [H, W] = [224, 224]

        mask = torch.nn.functional.interpolate(
            mask.float().unsqueeze(0).unsqueeze(0),  # Добавляем batch и channel [1, 1, 224, 224]
            size=(56, 56),  # Новый размер
            # size=(112, 112),  # Новый размер
            mode='nearest'  # Без интерполяции (сохраняем целые классы)
        ).squeeze().long()  # Убираем batch и channel -> [56, 56]


        # import matplotlib.pyplot as plt
        # print(mask.sum())
        # print(img_path)
        # plt.imshow(mask.cpu().numpy())
        # plt.show()


        return image, mask.long()

# Трансформации
transform = A.Compose([
    A.Resize(params["INPUT_SIZE"][0], params["INPUT_SIZE"][1]),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# --- 3. Модель FOMO ---
class FomoBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        self.mobilenet = mobilenet_v2(pretrained=True).features[:params['trunkAt']]  # Обрезаем MobileNetV2

    def forward(self, x):
        return self.mobilenet(x)


class FomoHead(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Увеличиваем глубину feature map перед классификацией
        self.conv1 = nn.Conv2d(24, 48, kernel_size=3, padding=1)  # 24 -> 48 каналов
        self.act1 = nn.ReLU()

        # Дополнительный свёрточный слой (опционально)
        self.conv2 = nn.Conv2d(48, 32, kernel_size=3, padding=1)  # 48 -> 32
        self.act2 = nn.ReLU()

        # Финал: 1x1 свёртка для классификации
        self.conv3 = nn.Conv2d(32, num_classes, kernel_size=1)  # [num_classes, 56, 56]

    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.conv3(x)
        return x

class FomoModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = FomoBackbone()
        self.head = FomoHead(num_classes)

    def forward(self, x):
        features = self.backbone(x)
        return self.head(features)

# --- 4. Обучение ---
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, masks in tqdm(dataloader, desc="Training"):
        images = images.to(device)
        masks = masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(dataloader)


# --- 5. Основной цикл ---
def main():
    # Загрузка данных
    train_dataset = CocoDataset(TRAIN_ANNOTATION_FILE, TRAIN_IMAGE_DIR, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=params["BATCH_SIZE"], shuffle=True)

    val_dataset = CocoDataset(VAL_ANNOTATION_FILE, TRAIN_IMAGE_DIR, transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=params["BATCH_SIZE"], shuffle=True)

    # Модель и оптимизатор
    model = FomoModel(params["NUM_CLASSES"]).to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=params["LR"])

    # Обучение
    for epoch in range(1, params["EPOCHS"]+1):
        epoch_start_time = time.time()
        train_loss = train(model, train_loader, criterion, optimizer, DEVICE)
        print(f"Epoch {epoch+1}/{params['EPOCHS']}, Loss: {train_loss:.4f}")

        # Логирование метрик
        task.get_logger().report_scalar(
            title="Loss", series="Train", value=train_loss.item(), iteration=epoch
        )

        if epoch != 0 and epoch%10 ==0:
            # Сохранение весов
            torch.save(model.state_dict(), f"FOMO_56_crossEntropy_{epoch}e_model_weights.pth")
            print("Model weights saved!")



print("prepare DONE")

# if __name__ == "__main__":
#     main()

NameError: name 'task' is not defined

In [31]:
main()

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


C:\Users\ILYA\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

C:\Users\ILYA\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.

Connecting multiple input models with the same name: `mobilenet_v2-b0353104`. This might result in the wrong model being used when executing remotely
Training: 100%|██████████| 942/942 [00:12<00:00, 76.33it/s]


Epoch 2/50, Loss: 0.0211


Training: 100%|██████████| 942/942 [00:12<00:00, 78.47it/s]


Epoch 3/50, Loss: 0.0137


...

Epoch 49/50, Loss: 0.0085


Training: 100%|██████████| 942/942 [00:11<00:00, 83.38it/s]


Epoch 50/50, Loss: 0.0085


Training: 100%|██████████| 942/942 [00:11<00:00, 83.72it/s]


Epoch 51/50, Loss: 0.0084
Model weights saved!
